# 约束优化

## 目标和预备知识

如果您想提高建模技能，不妨尝试这个棘手的约束优化问题。我们将向您展示如何使用Gurobi Python API将此问题建模为线性规划问题，并使用Gurobi优化器求解。

这个模型是H. Paul Williams所著《Model Building in Mathematical Programming》第五版第273页和328-330页中的示例18。

这是一个高级建模示例，我们假设您了解Python和Gurobi Python API，并且具有构建数学优化模型的高级知识。通常，这些示例的目标函数和/或约束条件比较复杂，或需要使用Gurobi Python API的高级功能。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。


---
## 问题描述

在一个整数规划模型中，出现了以下约束：

$$
9x_{1} + 13x_{2} -14x_{3} + 17x_{4} + 13x_{5} - 19x_{6} + 23x_{7} + 21x_{8} \leq 37
$$

约束中的所有决策变量都是二进制变量，目标是找到另一个包含相同二进制变量的约束，该约束在逻辑上等价于原始约束，但具有最小可能的右侧值(RHS)的绝对值。

在整数规划理论中，求解整数规划问题的效率取决于线性规划松弛与整数解的凸包之间的"接近"程度。当这个表述"接近"整数解的凸包时，线性规划松弛是"紧的"。我们说，相对于原始约束的等价约束是"更紧的"，当等价约束具有与原始约束相同的整数解，但移除了原始约束的一些分数可行解。下图说明了这些概念。

![OptimizedConstraint](OptimizedConstraint.PNG)

图中的原始约束由线AB确定，而"更紧的"等价约束由线CD定义。注意，等价约束具有相同的整数解集，并且移除了原始约束的分数可行解。


---
## 模型构建

Bradley等人(1974)描述了一个简化单个0-1约束的程序。我们采用他们使用线性规划模型的程序。考虑以降序排列的正系数的标准形式约束是很方便的。这可以通过以下变换实现：

$$
y_{1} = x_{7}, y_{2} = x_{8}, y_{3} = 1 - x_{6}, y_{4} = x_{4},
$$

$$
y_{5} = 1 - x_{3}, y_{6} = x_{5}, y_{7} = x_{2}, y_{8} = x_{1}.
$$

得到的等价约束是：

$$
23y_{1} + 21y_{2} + 19y_{3} + 17y_{4} + 14y_{5} + 13y_{6} + 13y_{7} + 9y_{8} \leq 70.
$$

形如$\sum_{i=1}^{8} b_{i}*y_{i} \leq b_{0}$的所有等价于待优化约束的不等式都可以用一组以决策变量$b_{i}$表示的线性规划模型的不等式来表征。注意，变量$b_{i}$是变换后约束的系数。

为了捕捉原始约束的完整逻辑含义，我们搜索称为*roofs*和*ceilings*的索引子集。Ceilings是变量索引的"最大"子集，其对应系数之和不超过右侧系数。这样的子集是最大的，因为没有包含它的子集，或者在隐含的字典序中位于左侧的子集，也可以是ceiling。例如，子集{1, 2, 4, 8}是一个ceiling，$23 +21 + 17 + 9 \leq 70$，但任何包含它的子集(例如{1, 2, 4, 7, 8})或其"左侧"的子集(例如{1, 2, 4, 7})都不是ceiling。Roofs是"最小"的索引子集，其对应系数之和超过右侧系数。这种子集以相同的意义是"最小的"。例如{2, 3, 4, 5}是一个roof，
$21 +19+ 17 + 14 > 70$，
但任何包含在它内部的子集(例如{3, 4, 5})或其"右侧"的子集(例如
{2, 3, 4, 6})都不是roof。有关这些约束如何推导的详细信息，请参见Bradley等人(1974)。

因此，形如$\sum_{i=1}^{8} b_{i}*y_{i} \leq b_{0}$的所有等价于待优化约束的不等式都可以用以下约束来表征。

**Ceiling约束**: <br />

$$
\{1,2,3 \}: b_{1} + b_{2} + b_{3} \leq b_{0}
$$

$$
\{1,2,4,8 \}: b_{1} + b_{2} + b_{4} + b_{8}  \leq b_{0}
$$

$$
\{1,2,6,7 \}: b_{1} + b_{2} + b_{6} + b_{7}  \leq b_{0}
$$

$$
\{1,3,5,6 \}: b_{1} + b[3] + b[5] + b[6]  \leq b_{0}
$$

$$
\{2,3,4,6 \}: b_{2} + b_{3} + b_{4} + b_{6}  \leq b_{0}
$$

$$
\{2,5,6,7,8 \}: b_{2} + b_{5} + b_{6} + b_{7} + b_{8} \leq b_{0}
$$

**Roof约束**: <br />

$$
\{1,2,3,8 \}: b_{1} + b_{2} + b_{3} + b_{8} \geq b_{0} + 1
$$

$$
\{1,2,5,7 \}: b_{1} + b_{2} + b_{5} + b_{7}  \geq b_{0} + 1
$$

$$
\{1,3,4,7 \}: b_{1} + b_{3} + b_{4} + b_{7} \geq b_{0} + 1
$$

$$
\{1,5,6,7,8 \}: b_{1} + b_{5} + b_{6} + b_{7} + b_{8} \geq b_{0} + 1
$$

$$
\{2,3,4,5 \}: b_{2} + b_{3} + b_{4} + b_{5} \geq b_{0} + 1
$$

$$
\{3,4,6,7,8 \}: b_{3} + b_{4} + b_{6} + b_{7} + b_{8} \geq b_{0} + 1
$$


$$
\text{Decreasing value:} \quad b_{i} \geq b_{i+1} \quad \forall i=1..7
$$

$$
\text{Non-negativity:} \quad b_{i} \geq 0 \quad \forall i=0,1..7
$$

目标是最小化原始约束右侧的值。
$$
\text{Objective function:} \quad \text{Min} \;\; (b_{0} - b_{3} - b_{5} )
$$

---
## Python实现

我们导入Gurobi Python模块。

In [ ]:
%pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 测试环境：Python 3.11 & Gurobi 11.0

In [ ]:
# 变量b[i]的索引列表

indices = [*range(9)]

## 模型部署
我们创建一个模型和决策变量。这些变量确定了转换后的等价约束的系数，它将最小化RHS的值。

In [ ]:
model = gp.Model('ConstraintOptimization')

# 变量b[i]
b = model.addVars(indices, name="b")

Using license file c:\gurobi\gurobi.lic


### Ceiling约束

In [ ]:
# 约束 {1, 2, 3}:

C123 = model.addConstr( b[1] + b[2] + b[3] <= b[0] , name='C123')

# 约束 {1, 2, 4, 8}}:

C1248 = model.addConstr( b[1] + b[2] + b[4] + b[8] <= b[0] , name='C1248')

# 约束 {1, 2, 6, 7}:

C1267 = model.addConstr( b[1] + b[2] + b[6] + b[7] <= b[0] , name='C1267')

# 约束 {1, 3, 5, 6}:

C1356 = model.addConstr( b[1] + b[3] + b[5] + b[6] <= b[0] , name='C1356')

# 约束 {2, 3, 4, 6}:

C2346 = model.addConstr( b[2] + b[3] + b[4] + b[6] <= b[0] , name='C2346')

# 约束 {2, 5, 6, 7, 8}:

C25678 = model.addConstr( b[2] + b[5] + b[6] + b[7] + b[8] <= b[0] , name='C25678')

### Roof约束

In [ ]:
# 约束 {1, 2, 3, 8}:

R1238 = model.addConstr( b[1] + b[2] + b[3] + b[8] >= b[0] + 1 , name='R1238')

# 约束 {1, 2, 5, 7}:

R1257 = model.addConstr( b[1] + b[2] + b[5] + b[7] >= b[0] + 1 , name='R1257')

# 约束 {1, 3, 4, 7}:

R1347 = model.addConstr( b[1] + b[3] + b[4] + b[7] >= b[0] + 1 , name='R1347')

# 约束 {1, 5, 6, 7, 8}:

R15678 = model.addConstr( b[1] + b[5] + b[6] + b[7] + b[8] >= b[0] + 1 , name='R15678')

# 约束 {2, 3, 4, 5}:

R2345 = model.addConstr( b[2] + b[3] + b[4] + b[5] >= b[0] + 1 , name='R2345')

# 约束 {3, 4, 6, 7, 8}:

R34678 = model.addConstr( b[3] + b[4] + b[6] + b[7] + b[8] >= b[0] + 1 , name='R34678')

### 递减值系数
这些约束保证了系数的顺序。

In [ ]:
# 系数递减值约束

DV1_2 = model.addConstr( b[1] >= b[2] , name='DV1_2')
DV2_3 = model.addConstr( b[2] >= b[3] , name='DV2_3')
DV3_4 = model.addConstr( b[3] >= b[4] , name='DV3_4')
DV4_5 = model.addConstr( b[4] >= b[5] , name='DV4_5')
DV5_6 = model.addConstr( b[5] >= b[6] , name='DV5_6')
DV6_7 = model.addConstr( b[6] >= b[7] , name='DV6_7')
DV7_8 = model.addConstr( b[7] >= b[8] , name='DV7_8')

目标是最小化原始约束右侧的值。

In [ ]:
# 目标函数

model.setObjective(b[0] -b[3] -b[5] )

In [ ]:
# 验证模型构建

model.write('OptimizeConstraint.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 19 rows, 9 columns and 76 nonzeros
Model fingerprint: 0x88e2e477
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 19 rows, 9 columns, 76 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+30   7.000000e+30   2.000000e+00      0s
      16    2.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  2.500000000e+01


In [ ]:
# 输出报告


# 相对于原始变量x的优化系数值

a = {}

a[0] = b[0].x - b[3].x - b[5].x
a[1] = b[8].x
a[2] = b[7].x
a[3] = -b[5].x
a[4] = b[4].x
a[5] = b[6].x
a[6] = -b[3].x
a[7] = b[1].x
a[8] = b[2].x

print("________________________________________")
print(f"等价优化约束为:")


optimized_constraint = {}
for i in indices:
    if i==0:
        optimized_constraint[i]=f"{a[i]}"
    else:
        optimized_constraint[i]= f"{a[i]}x[{i}]"
        #print(optimized_constraint)
            
print(f"\n{optimized_constraint[1]} + {optimized_constraint[2]} + {optimized_constraint[3]} + {optimized_constraint[4]} + {optimized_constraint[5]} + {optimized_constraint[6]}  + {optimized_constraint[7]} + {optimized_constraint[8]} <= {optimized_constraint[0]}")

________________________________________
The equivalent optimized constraint is:

6.0x[1] + 9.0x[2] + -10.0x[3] + 12.0x[4] + 9.0x[5] + -13.0x[6]  + 16.0x[7] + 14.0x[8] <= 25.0


---
## 参考文献

H. Paul Williams著《Model Building in Mathematical Programming》第五版。

Bradley, G.H., Hammer, P.L. 和 Wolsey, L. (1974) 0-1变量不等式的系数简化。Mathematical Programming, 7, 263-282。

版权所有 © 2020 Gurobi Optimization, LLC